In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

In [2]:
article = 'Green_Day'


def dataExtracting(elem, url, edges, urls, visited):
    href = elem['href']
    url = url.split('/')[-1].split(':')[-1]
    title = href.split(':')[-1]
    edges.append([dataCleaning(url), dataCleaning(title)])
    if href not in urls and href not in visited:
        urls.append(href)

def dataCleaning(title):
    return title.replace('_', ' ').replace('%C3%B6', 'ö').replace('%C3%9C', 'Ü').replace('%C3%A4', 'ä').replace('%C3%BC', 'ü').replace('%C3%9F', 'ß')

In [3]:
start = time.time()

main_url = 'https://de.wikipedia.org'
html = urlopen(main_url+'/wiki/'+article)
bs = BeautifulSoup(html, 'lxml')

edges = []
urls_next = []
urls_visited = []
for elem in bs.find('div', {'id':'mw-normal-catlinks'}).findAll('a')[1:]:
    dataExtracting(elem, main_url, edges, urls_next, urls_visited)
    
while True:
    try:
        url = urls_next.pop()
    except IndexError:
        break
    
    html = urlopen(main_url+url)
    bs = BeautifulSoup(html, 'lxml')
    urls_visited.append(url)
    
    try:
        for elem in bs.find('div', {'id':'mw-normal-catlinks'}).findAll('a')[1:]:
            dataExtracting(elem, url, edges, urls_next, urls_visited)
    except:
        pass
    
ende = time.time()
print('Dauer:\t{}'.format(ende-start))
print('Edges:\t{}'.format(len(edges)))
print('VURLs:\t{}'.format(len(urls_visited)))
print('NURLs:\t{}'.format(len(urls_next)))
print('URLs/s:\t{}'.format(len(urls_visited)/(ende-start)))

Dauer:	40.04221177101135
Edges:	179
VURLs:	104
NURLs:	0
URLs/s:	2.5972591273115193


In [4]:
df = pd.DataFrame(edges, columns=['Source', 'Target']).drop_duplicates().reset_index(drop=True)
df.to_csv('build/Wiki_Categories_{}.csv'.format(article), index=False, encoding='utf-8')
df.shape

(179, 2)